In [ ]:
# !pip -q install tabula

In [1]:
import tabula
import pandas as pd
import os
import re
import numpy as np

In [2]:
# Function to extract date and identifier from the file name
def extract_date_and_identifiers(filename):
    # Adjust the regex to handle extra spaces around the hyphens
    match = re.search(r'-\s*([\d\s-]+)\s*\.pdf$', filename)
    if match:
        return match.group(1).replace(" ", "")  # Remove any extra spaces within the match
    return None

# Function to clean text
def clean_text(text):
    return re.sub(r'[^0-9.]', '', text)

# Define the labeling function
def label_company_energy(value):
    if value in ['hydros', 'kengen geothermal', 'muhoroni gt', 'ngong wind']:
        return 'KenGen'
    elif value in ['net uetcl import']:
        return 'UETCL Eng'
    else:
        return value
    
# Define the labeling function that acumulated only kengen plants
def label_company_thermal(value):
    if value in ['muhoroni gt', 'kdp-i', 'kdp-iii']:
        return 'KenGen'
    elif value in ['rabai power', 'iberafrica- plant ii,','thika power1','triumph power2', 'gulf power']:
        return 'Thermal (Non KenGen)'
    elif value in ['uetcl imp.']:
        return 'UETCL Ther'
    else:
        return np.nan

# Replace specific values in the Power Plant column
replacement_dict = {
    'rabai power2': 'rabai power',
    'iberafric a 2': 'iberafrica- plant ii,',
    'iberafrica 2': 'iberafrica- plant ii,',
    'kdp 3': 'kdp-iii',
    'muhoron i gt': 'muhoroni gt',
    'thika power': 'thika power1',
    'thika power2': 'thika power1',
    'triumph': 'triumph power2',
    'uetcl import': 'uetcl imp.'
}

# Initialize an empty dataframe to store the concatenated data
energy = []
major_hydros = []
thermal_uetcl = []

# Directory containing the PDF files
pdf_directory = 'Raw Data/Generation Data New/'

## Energy Table Data Extraction

In [17]:
# Initialize an empty dataframe to store the concatenated data
energy = []

# uploaded data
final_df_old = pd.read_csv('Final Data/ENERGY SUPPLY.csv')

# Iterate through each PDF file in the directory
for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_directory, filename)
        
        # Extract date and identifiers from the file name
        date_and_identifiers = extract_date_and_identifiers(filename)
        print(f'Processing {filename}: Date and identifiers: {date_and_identifiers}')
        
        # Initialize lists to store dataframes for each page
        page_dataframes = []

        # Read Page 1 with lattice=True
        tables_page1 = tabula.read_pdf(pdf_path, 
                                       pages=1, 
                                       multiple_tables=True, 
                                       lattice=True, 
                                       stream=True)
        
        if len(tables_page1) >= 1:
            if len(tables_page1[0]) > 1:
                Energy = tables_page1[0].iloc[1:16, 0:3].dropna()
                
            elif len(tables_page1[0]) == 1:
                Energy = tables_page1[1].iloc[1:16, 0:3].dropna()

            Energy.rename(columns={'Generation\rSource': "Power Plant",
                                'Generation': "Projected Dispatch (NCC Planning) (MWh)",
                                'Unnamed: 0': 'Actual Generation (system Control) (MWh)'}, inplace=True)
            
            print("Number of rows in Energy Supply table:", len(Energy))
            # Add date and identifiers column
            Energy['Date'] = date_and_identifiers
            # Energy['Report'] = "ENERGY SUPPLY"
            page_dataframes.append(Energy)
        
        # Concatenate the dataframes from all pages for the current file
        combined_df = pd.concat(page_dataframes, ignore_index=True)
        
        # Append the combined dataframe to the list
        energy.append(combined_df)

# Concatenate all dataframes in the list into one dataframe
final_df_energy = pd.concat(energy, ignore_index=True)

# Replace \r with a space in 'Power Plant'
final_df_energy['Power Plant'] = final_df_energy['Power Plant'].apply(lambda x: x.replace('\r', ' ') if isinstance(x, str) else x)

# Convert Power Plant to lowercase
final_df_energy['Power Plant'] = final_df_energy['Power Plant'].str.lower()

# Apply the function to clean 
final_df_energy['Actual Generation (system Control) (MWh)'] = final_df_energy['Actual Generation (system Control) (MWh)'].apply(clean_text)

# Convert to numeric values
final_df_energy['Actual Generation (system Control) (MWh)'] = pd.to_numeric(final_df_energy['Actual Generation (system Control) (MWh)'])
final_df_energy['Projected Dispatch (NCC Planning) (MWh)'] = pd.to_numeric(final_df_energy['Projected Dispatch (NCC Planning) (MWh)'])

# Apply the function to create a new column 'Company'
final_df_energy['Company'] = final_df_energy['Power Plant'].apply(label_company_energy)

# Convert the 'Date' column to datetime
final_df_energy['Date'] = pd.to_datetime(final_df_energy['Date'], format='%d-%m-%Y')

final_df_energy = final_df_energy[[ 'Date', 'Company', 'Power Plant', 'Projected Dispatch (NCC Planning) (MWh)', 'Actual Generation (system Control) (MWh)']]

# Convert the 'Date' column to datetime
final_df_old['Date'] = pd.to_datetime(final_df_old['Date'], format='%Y-%m-%d')

# Merge new data with old data
final_df_energy = pd.concat([final_df_old, final_df_energy], axis=0, ignore_index=True)

# Display the length of final concatenated dataframe
len(final_df_energy)

Processing Daily  Power  Generation and Dispatch Analysis Report - 01- 08 - 2024.pdf: Date and identifiers: 01-08-2024
Number of rows in Energy Supply table: 12
Processing Daily  Power  Generation and Dispatch Analysis Report - 01- 09 - 2024.pdf: Date and identifiers: 01-09-2024
Number of rows in Energy Supply table: 12
Processing Daily  Power  Generation and Dispatch Analysis Report - 02- 08 - 2024.pdf: Date and identifiers: 02-08-2024
Number of rows in Energy Supply table: 12
Processing Daily  Power  Generation and Dispatch Analysis Report - 02- 09 - 2024.pdf: Date and identifiers: 02-09-2024
Number of rows in Energy Supply table: 12
Processing Daily  Power  Generation and Dispatch Analysis Report - 03- 08 - 2024.pdf: Date and identifiers: 03-08-2024
Number of rows in Energy Supply table: 12
Processing Daily  Power  Generation and Dispatch Analysis Report - 03- 09 - 2024.pdf: Date and identifiers: 03-09-2024
Number of rows in Energy Supply table: 12
Processing Daily  Power  Generatio

2772

In [14]:
path = "Raw Data/Generation Data New/Daily  Power  Generation and Dispatch Analysis Report - 31- 08 - 2024.pdf"
# Read Page 1 with lattice=True
tables_page1 = tabula.read_pdf(path, 
                                pages=1, 
                                multiple_tables=True, 
                                lattice=True, 
                                stream=True)

len(tables_page1[0])

1

## Major Hydros Table Data Extraction

In [37]:
# Initialize an empty dataframe to store the concatenated data
major_hydros = []

# uploaded data
final_df_old = pd.read_csv('Final Data/MAJOR HYDROS.csv')

# Iterate through each PDF file in the directory
for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_directory, filename)
        
        # Extract date and identifiers from the file name
        date_and_identifiers = extract_date_and_identifiers(filename)
        print(f'Processing {filename}: Date and identifiers: {date_and_identifiers}')
        
        # Initialize lists to store dataframes for each page
        page_dataframes = []

        # Read Page 2 with lattice=False
        tables_page2 = tabula.read_pdf(pdf_path, 
                                       pages=2, 
                                       multiple_tables=True, 
                                       lattice=False, 
                                       stream=True)
        
        if len(tables_page2) == 1:
            Major_hydros = tables_page2[0].iloc[5:, [1, 4, 5]].dropna()
            Major_hydros.rename(columns={'Unnamed: 1': "Power Plant",
                                         'Unnamed: 4': "Projected Dispatch (NCC Planning) (MWh)",
                                         'Unnamed: 5': 'Actual Generation (system Control) (MWh)'}, inplace=True)
            
            print("Number of rows in Major Hydros table:", len(Major_hydros))
            # Add date and identifiers column
            Major_hydros['Date'] = date_and_identifiers
            # Major_hydros['Report'] = "MAJOR HYDROS"
            page_dataframes.append(Major_hydros)
        
        # Concatenate the dataframes from all pages for the current file
        combined_df = pd.concat(page_dataframes, ignore_index=True)
        
        # Append the combined dataframe to the list
        major_hydros.append(combined_df)

# Concatenate all dataframes in the list into one dataframe
final_df_hydros = pd.concat(major_hydros, ignore_index=True)

# Convert to numeric values
final_df_hydros['Actual Generation (system Control) (MWh)'] = pd.to_numeric(final_df_hydros['Actual Generation (system Control) (MWh)'])
final_df_hydros['Projected Dispatch (NCC Planning) (MWh)'] = pd.to_numeric(final_df_hydros['Projected Dispatch (NCC Planning) (MWh)'])

# Convert the 'Date' column to datetime
final_df_hydros['Date'] = pd.to_datetime(final_df_hydros['Date'], format='%d-%m-%Y')

final_df_hydros = final_df_hydros[['Date', 'Power Plant', 'Projected Dispatch (NCC Planning) (MWh)', 'Actual Generation (system Control) (MWh)']]

# Convert the 'Date' column to datetime
final_df_old['Date'] = pd.to_datetime(final_df_old['Date'], format='%Y-%m-%d')

# Merge new data with old data
final_df_hydros = pd.concat([final_df_old, final_df_hydros], axis=0, ignore_index=True)

# Display the length of final concatenated dataframe
len(final_df_hydros)

Processing Daily  Power  Generation and Dispatch Analysis Report - 01- 08 - 2024.pdf: Date and identifiers: 01-08-2024
Number of rows in Major Hydros table: 9
Processing Daily  Power  Generation and Dispatch Analysis Report - 01- 09 - 2024.pdf: Date and identifiers: 01-09-2024
Number of rows in Major Hydros table: 9
Processing Daily  Power  Generation and Dispatch Analysis Report - 02- 08 - 2024.pdf: Date and identifiers: 02-08-2024
Number of rows in Major Hydros table: 9
Processing Daily  Power  Generation and Dispatch Analysis Report - 02- 09 - 2024.pdf: Date and identifiers: 02-09-2024
Number of rows in Major Hydros table: 9
Processing Daily  Power  Generation and Dispatch Analysis Report - 03- 08 - 2024.pdf: Date and identifiers: 03-08-2024
Number of rows in Major Hydros table: 9
Processing Daily  Power  Generation and Dispatch Analysis Report - 03- 09 - 2024.pdf: Date and identifiers: 03-09-2024
Number of rows in Major Hydros table: 9
Processing Daily  Power  Generation and Dispat

1566

## Thermal + UETCL Table Data Extraction

In [ ]:
# Initialize an empty dataframe to store the concatenated data
thermal_uetcl = []

# uploaded data
final_df_old = pd.read_csv('Final Data/THERMAL PLANTS + UETCL.csv')

# Iterate through each PDF file in the directory
for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_directory, filename)
        
        # Extract date and identifiers from the file name
        date_and_identifiers = extract_date_and_identifiers(filename)
        print(f'Processing {filename}: Date and identifiers: {date_and_identifiers}')
        
        # Initialize list to store dataframes for each page
        page_dataframes = []

        # Page 3 with lattice=True
        tables_page3 = tabula.read_pdf(pdf_path,
                                       pages=3,
                                       multiple_tables=True,
                                       lattice=True,
                                       stream=True)
        if len(tables_page3) > 0:
            try:
                Thermal_uetcl = tables_page3[0].loc[:10, ['Unnamed: 0', 'Unnamed: 5', 'Unnamed: 6']].dropna()
                # Apply the cleaning function to the column
                Thermal_uetcl['Unnamed: 6'] = Thermal_uetcl['Unnamed: 6'].apply(clean_text)
                # Renaming variables
                Thermal_uetcl.rename(columns={'Unnamed: 0': "Power Plant",
                                        'Unnamed: 5': "Projected Dispatch (NCC Planning) (MWh)",
                                        'Unnamed: 6': 'Actual Generation (system Control) (MWh)'}, inplace=True)
            except TypeError:
                Thermal_uetcl = tables_page3[0].loc[:10, ['Unnamed: 0', 'Unnamed: 6', 'Unnamed: 7']].dropna()
                # Apply the cleaning function to the column
                Thermal_uetcl['Unnamed: 7'] = Thermal_uetcl['Unnamed: 7'].apply(clean_text)
                # Renaming variables
                Thermal_uetcl.rename(columns={'Unnamed: 0': "Power Plant",
                                        'Unnamed: 6': "Projected Dispatch (NCC Planning) (MWh)",
                                        'Unnamed: 7': 'Actual Generation (system Control) (MWh)'}, inplace=True)
            except KeyError:
                Thermal_uetcl = tables_page3[1].loc[:11, ['Unnamed: 0', 'Unnamed: 5', 'Unnamed: 6']].dropna()
                # Apply the cleaning function to the column
                Thermal_uetcl['Unnamed: 6'] = Thermal_uetcl['Unnamed: 6'].apply(clean_text)
                # Renaming variables
                Thermal_uetcl.rename(columns={'Unnamed: 0': "Power Plant",
                                            'Unnamed: 5': "Projected Dispatch (NCC Planning) (MWh)",
                                            'Unnamed: 6': 'Actual Generation (system Control) (MWh)'}, inplace=True)
                
            print("Number of rows", len(Thermal_uetcl))
            
            # Add date and identifiers column
            Thermal_uetcl['Date'] = date_and_identifiers
            # Thermal_uetcl['Report'] = "THERMAL PLANTS + UETCL"
            page_dataframes.append(Thermal_uetcl)
        
        # Concatenate the dataframes from all pages for the current file
        combined_df = pd.concat(page_dataframes, ignore_index=True)
        
        # Append the combined dataframe to the list
        thermal_uetcl.append(combined_df)

# Concatenate all dataframes in the list into one dataframe
final_df_thermal = pd.concat(thermal_uetcl, ignore_index=True)

# Replace \r with a space in 'Power Plant'
final_df_thermal['Power Plant'] = final_df_thermal['Power Plant'].apply(lambda x: x.replace('\r', ' ') if isinstance(x, str) else x)
# Convert Power Plant to lowercase
final_df_thermal['Power Plant'] = final_df_thermal['Power Plant'].str.lower()
final_df_thermal['Power Plant'] = final_df_thermal['Power Plant'].replace(replacement_dict)

# Convert to numeric values
final_df_thermal['Actual Generation (system Control) (MWh)'] = pd.to_numeric(final_df_thermal['Actual Generation (system Control) (MWh)'])
final_df_thermal['Projected Dispatch (NCC Planning) (MWh)'] = pd.to_numeric(final_df_thermal['Projected Dispatch (NCC Planning) (MWh)'])

# Apply the function to create a new column 'Company'
final_df_thermal['Company'] = final_df_thermal['Power Plant'].apply(label_company_thermal)

# Convert the 'Date' column to datetime
final_df_thermal['Date'] = pd.to_datetime(final_df_thermal['Date'], format='%d-%m-%Y')

final_df_thermal = final_df_thermal[['Date', 'Company', 'Power Plant', 'Projected Dispatch (NCC Planning) (MWh)', 'Actual Generation (system Control) (MWh)']]

# Convert the 'Date' column to datetime
final_df_old['Date'] = pd.to_datetime(final_df_old['Date'], format='%Y-%m-%d')

# Merge new data with old data
final_df_thermal = pd.concat([final_df_old, final_df_thermal], axis=0, ignore_index=True)

# Display the length of final concatenated dataframe
len(final_df_thermal)

In [ ]:
path = "Daily  Power  Generation and Dispatch Analysis Report - 31- 07-2024.pdf" 
# Page 3 with lattice=True
tables_page3 = tabula.read_pdf(path,
                                pages=3,
                                multiple_tables=True,
                                lattice=True,
                                stream=True)

## Demand statistic table data extraction

In [45]:
# Empty List
Demand_stats = []

Lables = [
    'Date',
    'Actual available capacity at peak (MW)',
    'Day peak demand (MW)',
    'Highest peak demand (MW)',
    'Minimum demand (MW)',
    'System gross demand (MWh)'
]

# uploaded data
final_df_demand = pd.read_csv('Final Data/Demand Statistics.csv')

# Iterate through each PDF file in the directory
for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_directory, filename)
        
        # Extract tables from the PDF
        tables = tabula.read_pdf(pdf_path, 
                                pages='4-5', 
                                multiple_tables=True, 
                                lattice=True, 
                                stream=True,
                                pandas_options={'header': None}
                                )

        
        # Extract date and identifiers from the file name
        date_and_identifiers = extract_date_and_identifiers(filename)
        print(f'Processing {filename}: Date and identifiers: {date_and_identifiers}')

        # Print the number of tables detected
        print(f'Total tables detected: {len(tables)}')
        
        if len(tables) > 0:
            try:
                Demand_stat = tables[6].iloc[:, 0:2]

                 # Set new header
                Demand_stat.columns = Demand_stat.iloc[0]

                # Drop the rows above the new header
                Demand_stat = Demand_stat[1:]

                # Apply the cleaning function to the column
                Demand_stat['Figure'] = Demand_stat['Figure'].apply(clean_text)
                

            except IndexError:
                Demand_stat = tables[5].iloc[:, 0:2]

                 # Set new header
                Demand_stat.columns = Demand_stat.iloc[0]

                # Drop the rows above the new header
                Demand_stat = Demand_stat[1:]
                
                # Apply the cleaning function to the column
                Demand_stat['Figure'] = Demand_stat['Figure'].apply(clean_text)
                
            except KeyError:
                # concatnating two tables
                Demand_stat=pd.concat([tables[5], tables[6]], ignore_index=True, axis=0)

                # Set new header
                Demand_stat.columns = Demand_stat.iloc[0]

                # Drop the rows above the new header
                Demand_stat = Demand_stat[1:]
                Demand_stat = Demand_stat.iloc[:, 0:2]
            
                # Apply the cleaning function to the column
                Demand_stat['Figure'] = Demand_stat['Figure'].apply(clean_text)
            
            Demand_stat['Date'] = date_and_identifiers
            Demand_stat = Demand_stat.pivot(index='Date', columns='Parameter', values='Figure')

            # # Flatten the column multi-index if necessary
            Demand_stat.columns.name = None
            Demand_stat.reset_index(inplace=True)
            
            # Rename the columns in the DataFrame
            Demand_stat.columns = Lables
            
            # Append the combined dataframe to the list
            Demand_stats.append(Demand_stat)

# Concatenate all dataframes in the list into one dataframe
Demand_stats = pd.concat(Demand_stats, ignore_index=False)

# Convert the 'Date' column to datetime
Demand_stats['Date'] = pd.to_datetime(Demand_stats['Date'], format='%d-%m-%Y')

# setting date as index
Demand_stats.set_index('Date', inplace=True)

# Ensure all dates are present, filling with NaN for missing data
Demand_stats = Demand_stats.asfreq('D')

# Reset index
Demand_stats = Demand_stats.reset_index()

# Convert the 'Date' column to datetime
final_df_demand['Date'] = pd.to_datetime(final_df_demand['Date'], format='%Y-%m-%d')

# Merge new data with old data
Demand_stats = pd.concat([final_df_demand, Demand_stats], axis=0, ignore_index=True)

# Display the length of final concatenated dataframe
len(Demand_stats)


Processing Daily  Power  Generation and Dispatch Analysis Report - 01- 08 - 2024.pdf: Date and identifiers: 01-08-2024
Total tables detected: 7
Processing Daily  Power  Generation and Dispatch Analysis Report - 01- 09 - 2024.pdf: Date and identifiers: 01-09-2024
Total tables detected: 7
Processing Daily  Power  Generation and Dispatch Analysis Report - 02- 08 - 2024.pdf: Date and identifiers: 02-08-2024
Total tables detected: 7
Processing Daily  Power  Generation and Dispatch Analysis Report - 02- 09 - 2024.pdf: Date and identifiers: 02-09-2024
Total tables detected: 7
Processing Daily  Power  Generation and Dispatch Analysis Report - 03- 08 - 2024.pdf: Date and identifiers: 03-08-2024
Total tables detected: 7
Processing Daily  Power  Generation and Dispatch Analysis Report - 03- 09 - 2024.pdf: Date and identifiers: 03-09-2024
Total tables detected: 7
Processing Daily  Power  Generation and Dispatch Analysis Report - 04- 08 - 2024.pdf: Date and identifiers: 04-08-2024
Total tables dete

242

In [113]:
# Combining Energy supply table with thermal table
Energy_dispatched = pd.concat([final_df_energy, final_df_thermal], ignore_index=True, axis=0)

# Categories to remove
categories_to_remove = ['thermal (diesel)', 'UETCL Ther']

# Filter out unwanted categories
Energy_dispatched = Energy_dispatched[~Energy_dispatched['Company'].isin(categories_to_remove)]

# Summing up the groups by company and date
Energy_dispatched = Energy_dispatched.groupby(['Date','Company'])[['Projected Dispatch (NCC Planning) (MWh)','Actual Generation (system Control) (MWh)']].sum().reset_index()


In [47]:
# Uploading data to csv file
# final_df_energy.to_csv('Final Data/ENERGY SUPPLY.csv', index=False)
# final_df_hydros.to_csv('Final Data/MAJOR HYDROS.csv', index=False)
final_df_thermal.to_csv('Final Data/THERMAL PLANTS + UETCL.csv', index=False)
# Demand_stats.to_csv('Final Data/Demand Statistics.csv', index=False)
Energy_dispatched.to_csv('Final Data/Energy Dispatch Summary.csv', index=False)

In [46]:
Demand_stats

,Date,Actual available capacity at peak (MW),Day peak demand (MW),Highest peak demand (MW),Minimum demand (MW),System gross demand (MWh),Day (0600-1700hrs) peak demand (MW),Daytime (6am-5pm) peak demand (MW)
0,2024-07-17,2356.0,1800.0,2228.0,1213.0,40541.0,NaN,NaN
1,2024-07-16,2217.0,1805.0,2148.0,1191.0,39889.0,NaN,NaN
2,2024-07-15,2166.0,1634.0,2101.0,1029.0,37311.0,NaN,NaN
3,2024-07-14,2365.0,1555.0,1988.0,1158.0,34652.0,NaN,NaN
4,2024-07-13,2387.0,1783.0,2094.0,1227.0,38686.0,NaN,NaN
...,...,...,...,...,...,...,...,...
237,2024-09-09,2268,1773.58,2157.60,1011.00,37632.62,NaN,NaN
238,2024-09-10,2433,1813.52,2219.13,1243.39,40507.41,NaN,NaN
239,2024-09-11,2431,1800.30,2232.33,1220.94,40312.07,NaN,NaN
240,2024-09-12,2508,1849.93,2185.45,1246.79,40503.23,NaN,NaN
